# NFL Passing

### Unindo e realizando as transformações(Limpeza) necessarias

### Bibliotecas Utilizadas:

In [1]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Manipulação de pastas e arquivos
import os
import glob

# Ignore Warning(Avisos)
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

### Carregando e unindo os arquivos csv

In [2]:
# Selecionando a extensão dos arquivos
extension = 'csv'

# Unindo os arquivos CSVs em um DataFrame
all_filenames = [i for i in glob.glob(f'../../DATA/DATA_FOR_EXTRACT/PASSING/*'.format(extension))]
data = pd.concat([pd.read_csv(f) for f in all_filenames ])

### Informações do dataset

In [3]:
# Verificando o nome das colunas do DataFrame
data.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%',
       'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C',
       'Y/G', 'Rate', 'QBR', 'Sk', 'Yds.1', 'Sk%', 'NY/A', 'ANY/A', '4QC',
       'GWD', 'Year'],
      dtype='object')

In [4]:
# Verificando a quantidade de linhas e colunas do DataFrame
data.shape

(1290, 31)

In [5]:
# Info do dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1290 entries, 0 to 77
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  1255 non-null   object 
 1   Tm      1255 non-null   object 
 2   Age     1255 non-null   float64
 3   Pos     1229 non-null   object 
 4   G       1255 non-null   float64
 5   GS      1255 non-null   float64
 6   QBrec   709 non-null    object 
 7   Cmp     1255 non-null   float64
 8   Att     1255 non-null   float64
 9   Cmp%    1255 non-null   float64
 10  Yds     1255 non-null   float64
 11  TD      1255 non-null   float64
 12  TD%     1255 non-null   float64
 13  Int     1255 non-null   float64
 14  Int%    1255 non-null   float64
 15  1D      1255 non-null   float64
 16  Lng     1255 non-null   float64
 17  Y/A     1255 non-null   float64
 18  AY/A    1255 non-null   float64
 19  Y/C     1097 non-null   float64
 20  Y/G     1255 non-null   float64
 21  Rate    1255 non-null   float64
 22  QB

### Verificando valores ausentes / nulos

In [6]:
# Calcula o total de células no dataset
totalCells = np.product(data.shape)

# Conta o número de valores ausentes por coluna
missingCount = data.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

O Dataset tem 8.62 % de valores ausentes.


In [7]:
# Total de valores ausentes
mis_val = data.isnull().sum()

# Porcentagem de valores ausentes
mis_val_percent = 100 * mis_val / len(data)

# Tipo de dado das colunas com valores ausentes
mis_val_dtype = data.dtypes

# Cria uma tabela com os resultados
mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_dtype], axis=1)

# Renomear as colunas
mis_val_table_ren_columns = mis_val_table.rename(
columns = {0 : 'Valores Ausentes', 1 : '% de Valores Ausentes', 2: 'Dtype'})

# Classifica a tabela por porcentagem de valores ausentes de forma decrescente e remove colunas sem valores faltantes
mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,0] != 0].sort_values('% de Valores Ausentes', ascending = False).round(2)

# Print 
print ("O dataset tem " + str(data.shape[1]) + " colunas.\n"
        "Encontrado: " + str(mis_val_table_ren_columns.shape[0]) + " colunas que têm valores ausentes.")

# Retorna o dataframe com informações ausentes
mis_val_table_ren_columns

O dataset tem 31 colunas.
Encontrado: 30 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
GWD,820,63.57,float64
4QC,820,63.57,float64
QBrec,581,45.04,object
Y/C,193,14.96,float64
QBR,133,10.31,float64
Pos,61,4.73,object
Lng,35,2.71,float64
ANY/A,35,2.71,float64
NY/A,35,2.71,float64
Sk%,35,2.71,float64


* Vamos adotar o seguinte procedimento para cada coluna:
- Coluna 'Unnamed: 0','QBrec','QBR','GWD','4QC' -> Essas colunas serão eliminadas.
- Coluna 'Yds.1' -> Iremos renomear essa coluna para SYds (Sacks/Yards)
- Coluna 'Y/C' -> Será imputado o valor 0(Zero) para os valores Nulos
- Iremos eliminar os restantes das linhas que sobrarem com valores Nulos

In [8]:
# Eliminando colunas
data.drop(['QBrec','QBR','GWD','4QC'], axis=1, inplace=True)

# Renomeando coluna
data.rename(columns={'Yds.1':'SYds'}, inplace=True)

# Imputando na coluna 'Y/C' o valor 0(Zero) nos valores NaN
data['Y/C'].fillna('0.0', inplace=True)

# Removendo linhas com valores nulos
data.dropna(inplace=True)

In [9]:
# Calcula o total de células no dataset
totalCells = np.product(data.shape)

# Conta o número de valores ausentes por coluna
missingCount = data.isnull().sum()

# Calcula o total de valores ausentes
totalMissing = missingCount.sum()

# Calcula o percentual de valores ausentes
print("O Dataset tem", round(((totalMissing/totalCells) * 100), 2), "%", "de valores ausentes.")

O Dataset tem 0.0 % de valores ausentes.


### Removendo caracteres de algumas linhas no dataset

* Na coluna com o nome do jogador(Player) a alguns nomes com caracters

In [10]:
data[data['Player'].str.contains('Tom Brady', regex=True)].head(3)

,Player,Tm,Age,Pos,G,GS,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,1D,Lng,Y/A,AY/A,Y/C,Y/G,Rate,Sk,SYds,Sk%,NY/A,ANY/A,Year
7,Tom Brady*+,NWE,33.0,QB,16.0,16.0,324.0,492.0,65.9,3900.0,36.0,7.3,4.0,0.8,189.0,79.0,7.9,9.0,12.0,243.8,111.0,25.0,175.0,4.8,7.21,8.25,2010
1,Tom Brady*,NWE,34.0,QB,16.0,16.0,401.0,611.0,65.6,5235.0,39.0,6.4,12.0,2.0,262.0,99.0,8.6,9.0,13.1,327.2,105.6,32.0,173.0,5.0,7.87,8.25,2011
3,Tom Brady*,NWE,35.0,QB,16.0,16.0,401.0,637.0,63.0,4827.0,34.0,5.3,8.0,1.3,256.0,83.0,7.6,8.1,12.0,301.7,98.7,27.0,182.0,4.1,7.00,7.48,2012


In [11]:
# Eliminando os caracteres
data['Player'] = data['Player'].str.replace('*', '')
data['Player'] = data['Player'].str.replace('+', '')

# Verificando
data[data['Player'].str.contains('Tom Brady', regex=True)].head(3)

,Player,Tm,Age,Pos,G,GS,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,1D,Lng,Y/A,AY/A,Y/C,Y/G,Rate,Sk,SYds,Sk%,NY/A,ANY/A,Year
7,Tom Brady,NWE,33.0,QB,16.0,16.0,324.0,492.0,65.9,3900.0,36.0,7.3,4.0,0.8,189.0,79.0,7.9,9.0,12.0,243.8,111.0,25.0,175.0,4.8,7.21,8.25,2010
1,Tom Brady,NWE,34.0,QB,16.0,16.0,401.0,611.0,65.6,5235.0,39.0,6.4,12.0,2.0,262.0,99.0,8.6,9.0,13.1,327.2,105.6,32.0,173.0,5.0,7.87,8.25,2011
3,Tom Brady,NWE,35.0,QB,16.0,16.0,401.0,637.0,63.0,4827.0,34.0,5.3,8.0,1.3,256.0,83.0,7.6,8.1,12.0,301.7,98.7,27.0,182.0,4.1,7.00,7.48,2012


### Verificando e ajustando outras colunas do tipo 'object' = String

* Colunas: 
- 'Tm' -> Nome do Time
- 'Pos' -> Posição do jogador 

In [12]:
# Verificandos os dados da colunas 'Pos' - valores únicos
data['Pos'].unique()

array(['QB', 'qb', 'wr', 'WR', 'p', 'RB', 'rb', 'k', 'LCB', 'db', 'FS',
       'te', 'NT', 'lb', 'TE', 'wr/qb', 'P'], dtype=object)

In [13]:
# Colocando os dados da coluna 'Pos' com maiúsculas
data['Pos'] = data['Pos'].str.upper()

In [14]:
data['Pos'].unique()

array(['QB', 'WR', 'P', 'RB', 'K', 'LCB', 'DB', 'FS', 'TE', 'NT', 'LB',
       'WR/QB'], dtype=object)

In [15]:
# Verificando os dados únicos da coluna 'Tm'
data['Tm'].unique()

array(['SDG', 'IND', 'NOR', 'HOU', 'NYG', 'CIN', 'GNB', 'NWE', 'ATL',
       'DEN', 'BAL', 'STL', 'TAM', 'WAS', 'MIA', 'NYJ', 'CHI', 'PIT',
       'KAN', 'PHI', 'SEA', 'BUF', 'JAX', 'DET', 'MIN', 'OAK', 'SFO',
       'DAL', 'ARI', 'TEN', 'CLE', 'CAR', 'LAR', 'LAC', 'LVR'],
      dtype=object)

### Criando novas variáveis

In [16]:
data['SYds/G'] = round(data['SYds'] / data['G'], 2)
data['Sk/G'] = round(data['Sk'] / data['G'], 2)
data['TD/G'] = round(data['TD'] / data['G'], 2)
data['Int/G'] = round(data['Int'] / data['G'], 2)

### Alterando o tipo dos dados de algumas colunas

In [17]:
data['Age'] = data['Age'].astype(np.int64)
data['G'] = data['G'].astype(np.int64)
data['GS'] = data['GS'].astype(np.int64)
data['Cmp'] = data['Cmp'].astype(np.int64)
data['Att'] = data['Att'].astype(np.int64)
data['Yds'] = data['Yds'].astype(np.int64)
data['TD'] = data['TD'].astype(np.int64)
data['Int'] = data['Int'].astype(np.int64)
data['1D'] = data['1D'].astype(np.int64)
data['Lng'] = data['Lng'].astype(np.int64)
data['Sk'] = data['Sk'].astype(np.int64)
data['SYds'] = data['SYds'].astype(np.int64)

In [18]:
data.dtypes

Player     object
Tm         object
Age         int64
Pos        object
G           int64
GS          int64
Cmp         int64
Att         int64
Cmp%      float64
Yds         int64
TD          int64
TD%       float64
Int         int64
Int%      float64
1D          int64
Lng         int64
Y/A       float64
AY/A      float64
Y/C        object
Y/G       float64
Rate      float64
Sk          int64
SYds        int64
Sk%       float64
NY/A      float64
ANY/A     float64
Year        int64
SYds/G    float64
Sk/G      float64
TD/G      float64
Int/G     float64
dtype: object

### Reordenando o Data Frame

In [19]:
data = data[['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD/G', 'TD%', 'Int', 'Int/G', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'Sk', 'Sk/G', 'SYds', 'SYds/G', 'Sk%', 'NY/A', 'ANY/A', 'Year']]

In [20]:
data.head(2)

,Player,Tm,Age,Pos,G,GS,Cmp,Att,Cmp%,Yds,TD,TD/G,TD%,Int,Int/G,Int%,1D,Lng,Y/A,AY/A,Y/C,Y/G,Rate,Sk,Sk/G,SYds,SYds/G,Sk%,NY/A,ANY/A,Year
0,Philip Rivers,SDG,29,QB,16,16,357,541,66.0,4710,30,1.88,5.5,13,0.81,2.4,234,59,8.7,8.7,13.2,294.4,101.8,38,2.38,227,14.19,6.6,7.74,7.77,2010
1,Peyton Manning,IND,34,QB,16,16,450,679,66.3,4700,33,2.06,4.9,17,1.06,2.5,253,73,6.9,6.8,10.4,293.8,91.9,16,1.00,91,5.69,2.3,6.63,6.48,2010


### Salvando os dados

In [ ]:
data.to_csv('../../DATA/DATA_FOR_CLEANING/PASSING/PASSING_CLEAR.csv', index=False)